In [1]:
import os
os.chdir('..')

In [2]:
%pwd

'c:\\Users\\anasm\\OneDrive\\Desktop\\Image Processing\\Hand Gesture Detector'

In [3]:
from src.utils import read_yaml, create_directories
from src.constants import CONFIG_FILE_PATH, PARAM_FILE_PATH

In [4]:
#add to entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_classes: int

In [5]:
#add to configuration
class ConfigManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAM_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.param=read_yaml(param_filepath)

        create_directories(self.config.artifacts_root)

    def get_base_model_configs(self)->PrepareBaseModelConfig:
        base_model_config=self.config.prepare_base_model

        model_config=PrepareBaseModelConfig(
            root_dir=Path(base_model_config.root_dir),
            base_model_path=Path(base_model_config.base_model_path),
            updated_base_model_path=Path(base_model_config.updated_base_model_path),
            params_image_size=self.param.IMAGE_SIZE,
            params_classes=self.param.CLASSES
        )
        
        return model_config

In [6]:
import tensorflow as tf
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config=config

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

    def create_model(self):
        vgg16=tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=tuple(self.config.params_image_size))
        vgg16.trainable=False
        input_layer=tf.keras.layers.Input(shape=self.config.params_image_size)
        x=vgg16(input_layer)
        x=tf.keras.layers.Flatten()(x)
        no=256*self.config.params_classes
        # x=tf.keras.layers.Dense(units=self.config.params_classes, activation= 'relu')(x)
        while (no>self.config.params_classes):
            x=tf.keras.layers.Dense(units=no, activation='relu')(x)
            x=tf.keras.layers.Dropout(0.2)(x)
            no=int(no/4)
        x=tf.keras.layers.Dense(units=self.config.params_classes, activation='softmax')(x)
        self.model=tf.keras.Model(inputs=input_layer, outputs=x)
        self.model.summary()
        self.save_model(self.config.updated_base_model_path, self.model)

In [7]:
con_generator=ConfigManager()
model_config=con_generator.get_base_model_configs()

yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Hand Gesture Detector\config\config.yaml loaded successfully
yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Hand Gesture Detector\config\params.yaml loaded successfully


In [8]:
p=PrepareBaseModel(model_config)
p.create_model()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2560)           │    64,227,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 640)            │     1,639,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 160)            │       102,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40)             │         6,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,690,978 (307.81 MB)

 Trainable params: 65,976,290 (251.68 MB)

 Non-trainable params: 14,714,688 (56.13 MB)